In [1]:
# Data Import and Pre-Processing
import pandas as pd

In [21]:
train_words_raw = pd.read_csv('./data/train_notypes', sep='\t', header=None, names=['word', 'label'])
train_vocab = set(train_words_raw.word.unique())

In [25]:
import regex as re
from collections import namedtuple
class RegexFeatures(object):
    PATTERNS = {
        "isInitCapitalWord": re.compile(r'^[A-Z][a-z]+'),
        "isAllCapitalWord": re.compile(r'^[A-Z][A-Z]+$'),
        "isAllSmallCase": re.compile(r'^[a-z]+$'),
        "isWord": re.compile(r'^[a-zA-Z][a-zA-Z]+$'),
        "isAlphaNumeric": re.compile(r'^\p{Alnum}+$'),
        "isSingleCapLetter": re.compile(r'^[A-Z]$'),
        "containsDashes": re.compile(r'.*--.*'),
        "containsDash": re.compile(r'.*\-.*'),
        "singlePunctuation": re.compile(r'^\p{Punct}$'),
        "repeatedPunctuation": re.compile(r'^[\.\,!\?"\':;_\-]{2,}$'),
        "singleDot": re.compile(r'[.]'),
        "singleComma": re.compile(r'[,]'),
        "singleQuote": re.compile(r'[\']'),
        "isSpecialCharacter": re.compile(r'^[#;:\-/<>\'\"()&]$'),
        "fourDigits": re.compile(r'^\d\d\d\d$'),
        "isDigits": re.compile(r'^\d+$'),
        "isNumber": re.compile(r'^((\p{N}{,2}([,]?\p{N}{3})+)(\.\p{N}+)?)$'),
        "containsDigit": re.compile(r'.*\d+.*'),
        "endsWithDot": re.compile(r'\p{Alnum}+\.$'),
        "isURL": re.compile(r'^http[s]?://'),
        "isMention": re.compile(r'^(RT)?@[\p{Alnum}_]+$'),
        "isHashtag": re.compile(r'^#\p{Alnum}+$'),
        "isMoney": re.compile(r'^\$((\p{N}{,2}([,]?\p{N}{3})+)(\.\p{N}+)?)$'),
    }
    def __init__(self):
        print("Initialized RegexFeature")
    @staticmethod
    def process(word):
        features = dict()
        for k, p in RegexFeatures.PATTERNS.iteritems():
            if p.match(word):
                features[k] = True
        return features
    
    
def classification_report_to_df(report):
    report_list = []
    for i, line in enumerate(report.split("\n")):
        if i == 0:
            report_list.append(["class", "precision", "recall", "f1-score", "support"])
        else:
            line = line.strip()
            if line:
                if line.startswith("avg"):
                    line = line.replace("avg / total", "avg/total")
                line = re.split(r'\s+', line)
                report_list.append(tuple(line))
    return pd.DataFrame(report_list[1:], columns=report_list[0])


DATA_DIR="data/data/"
CLEANED_DIR="data/cleaned/"

Tag = namedtuple("Tag", ["token", "tag"])

def load_sequences(filename, sep="\t", notypes=False, test_data=False):
    sequences = []
    with open(filename) as fp:
        seq = []
        for line in fp:
            line = line.strip()
            if line:
                line = line.split(sep)
                if test_data:
                    assert len(line) == 1
                    line.append("?")
                if notypes:
                    line[1] = line[1][0]
                seq.append(Tag(*line))
            else:
                sequences.append(seq)
                seq = []
        if seq:
            sequences.append(seq)
    return sequences


def load_vocab(filename):
    vocab = set()
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            vocab.add(line)
    return vocab      

    
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
        
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr)) 
        
        
def plot_cm(y_test, y_pred, labels=[], axis=1):
    labels_s = dict((k,i) for i,k in enumerate(labels))
    cm = np.zeros((len(labels), len(labels)))
    for i,j in zip(sum(y_test, []), sum(y_pred, [])):
        i = labels_s[i]
        j = labels_s[j]
        cm[i,j] += 1
    with plt.rc_context(rc={'xtick.labelsize': 12, 'ytick.labelsize': 12,
                       'figure.figsize': (16,14)}):
        sns.heatmap(cm * 100/ cm.sum(axis=axis, keepdims=True),
                    #cmap=sns.cubehelix_palette(n_colors=100, rot=-.4, as_cmap=True),
                    cmap="Greys",
                    xticklabels=labels,
                    yticklabels=labels)
        plt.ylabel("True labels")
        plt.xlabel("Predicted labels")
        title = "Precision Plot"
        if axis== 0:
            title = "Recall Plot"
        plt.title(title)
    print(cm.shape)
    return cm


def print_sequences(sequences, predictions, filename, test_data=False, notypes=False):
    with open(filename, "wb+") as fp:
        for seq, pred in zip(sequences, predictions):
            for t, p in zip(seq, pred):
                token, tag = t
                if tag[0] == "U":
                    tag = "B%s" % tag[1:]
                if tag[0] == "E":
                    tag = "I%s" % tag[1:]
                if p[0] == "U":
                    p = "B%s" % p[1:]
                if p[0] == "E":
                    p = "I%s" % p[1:]
                if notypes:
                    tag = tag[0]
                    p = p[0]
                if test_data:
                    line = "\t".join((token, p))
                else:
                    line = "\t".join((token, tag, p))
                print >> fp, line
            print >> fp, ""
    print("Done")

In [30]:
train_sequences = load_sequences("data/train", sep="\t", notypes=False)
dev_sequences = load_sequences("data/dev", sep="\t", notypes=False)
test_sequences = load_sequences("data/test", sep="\t", notypes=False)

In [50]:
all_sentences = [[t[0] for t in seq] for seq in (train_sequences+dev_sequences)]

In [51]:
training_data = [ (
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
) ]
training_data

[(['the',
   'wall',
   'street',
   'journal',
   'reported',
   'today',
   'that',
   'apple',
   'corporation',
   'made',
   'money'],
  ['B', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'I', 'O', 'O']),
 (['georgia', 'tech', 'is', 'a', 'university', 'in', 'georgia'],
  ['B', 'I', 'O', 'O', 'O', 'O', 'B'])]

In [61]:
def generate_input_data(filename):
    with open(filename) as f:
        data = f.readlines()

    input_data = list()
    sentence_word_list = list()
    sentence_word_tag_list = list()
    for line in data:
        splitted_line = line.split("\t")

        if splitted_line[0] == "\n":
            one_sentence_tuple = (sentence_word_list, sentence_word_tag_list)
            input_data.append(one_sentence_tuple)
            sentence_word_list = []
            sentence_word_tag_list = []
        else:
            sentence_word_list.append(splitted_line[0].strip())
            sentence_word_tag_list.append(splitted_line[1].strip())

    return input_data

In [48]:
from six import iteritems

def prepare_sequence(seq, to_ix):
    return np.array([to_ix[w] for w in seq])

# Make up some training data
training_data = generate_input_data("data/train")

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
ix_to_word = dict([(v, k) for k, v in iteritems(word_to_ix)])

char_to_ix = {}
for word in word_to_ix.keys():
    for char in word:
        if char not in char_to_ix:
            char_to_ix[char] = len(char_to_ix)
ix_to_char = dict([(v, k) for k, v in iteritems(char_to_ix)])

tag_to_ix = { "B": 0, "I": 1, "O": 2, LstmCrfTagger.START_TAG: 3, LstmCrfTagger.END_TAG: 4 }
ix_to_tag = dict([(v, k) for k, v in iteritems(tag_to_ix)])

def prepare_input(sentence, tags):
    sent_seq = prepare_sequence(sentence, word_to_ix)
    tags_seq = prepare_sequence(tags, tag_to_ix)
    word_seq = \
        tf.keras.preprocessing.sequence.pad_sequences(
            [prepare_sequence(word, char_to_ix) for word in sentence], 
            padding='post', 
            value=-1)
    word_len_seq = \
        np.apply_along_axis(
            lambda seq: next(i for i, j in enumerate(seq) if j < 0), 
            axis=1, 
            arr=np.c_[word_seq, np.ones((word_seq.shape[0], 1)) * -1])
    return sent_seq, tags_seq, word_seq, np.squeeze(word_len_seq)

In [ ]:
# Model Start

In [44]:
import tensorflow as tf
tf.reset_default_graph()

In [45]:
# Model Params

learning_rate = 0.01
train_epoch = 10000
input_size = 10
batch_size = 100
num_units = 512
num_classes = 5

In [127]:
# Placeholders

inputs = tf.placeholder(tf.float32, [None, None, input_size], name='inputs')
labels = tf.placeholder(tf.int32, [None, None], name='labels')
batch_seq_len = tf.placeholder(tf.int32)
org_seq_len = tf.placeholder(tf.int32, [None])

In [128]:
# Bi-LSTM Cell
with tf.name_scope("BiLSTM"):
    with tf.variable_scope('forward'):
        lstm_fw = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    with tf.variable_scope('backward'):
        lstm_bw = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    
    (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_fw,
                                                                     cell_bw=lstm_bw,
                                                                     inputs=inputs,
                                                                     sequence_length=org_seq_len,
                                                                     dtype=tf.float32,
                                                                     scope="BiLSTM")



outputs = tf.concat([output_fw, output_bw], axis=2)

# FC
W = tf.get_variable("W", [2 * num_units, num_classes], dtype=tf.float32)
b = tf.get_variable("b", [num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())

outputs_flat = tf.reshape(outputs, [-1, 2 * num_units])
pred = tf.matmul(outputs_flat, W) + b
scores = tf.reshape(pred, [-1, batch_seq_len, num_classes])

# CRF

log_loss, trans_params = tf.contrib.crf.crf_log_likelihood(scores, labels, org_seq_len)
loss = tf.reduce_mean(-log_loss)

# viterbi Seq, score
viterbi_seq, viterbi_score = tf.contrib.crf.crf_decode(scores, trans_params, org_seq_len)

# Train Ops
train_opt = tf.train.AdamOptimizer(learning_rate)
train_op = train_opt.minimize(loss)

# Saver
saver = tf.train.Saver()

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# Train into Session

# Take From Batch
batch_inputs = []
batch_labels = []
batch_seq_lengths = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(train_epoch):
        for batch_inputs, batch_labels, batch_len, batch_seq_lengths in batch(x_, y_, seq_len_train, batch_size, input_size):
            tf_viterbi_seq, _ = sess.run([viterbi_seq, train_op],
                                        feed_dict={inputs: batch_inputs,
                                                  labels: batch_labels,
                                                  batch_seq_len:batch_len,
                                                  org_seq_len: batch_seq_lengths})
        
            if i % 30 == 0:
                mask = (np.expand_dims(np.arrange(batch_len), axis=0) < np.expand_dims(batch_seq_lengths, axis=1))
                total_labels = np.sum(batch_seq_lengths)
                correct_labels = np.sum((batch_labels == tf_viterbi_seq)* mask)
                accuracy = 100.0 * correct_labels / float(total_labels)
                print("Epoch ", i, " Accuracy: ", accuracy)
        
    saver.save(sess, './model_crf')

In [ ]:
# Test Accuracy and Pred

# Take From Batch
batch_test = []
batch_test_labels = []
batch_seq_t_lengths = 0

with tf.Sesion() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(train_epoch):
        for batch_inputs, batch_labels, batch_len, batch_seq_t_lengths in batch(x_t, y_t, seq_len_test, batch_test_size, input_size):
            tf_viterbi_seq = sess.run(viterbi_seq,
                                        feed_dict={inputs: batch_inputs,
                                                  labels: batch_labels,
                                                  batch_seq_len:batch_len,
                                                  org_seq_len: batch_seq_t_lengths})
        
            
            mask = (np.expand_dims(np.arrange(batch_len), axis=0) < np.expand_dims(batch_seq_lengths, axis=1))
            total_labels = np.sum(batch_seq_t_lengths)
            correct_labels = np.sum((batch_labels == tf_viterbi_seq)* mask)
            accuracy = 100.0 * correct_labels / float(total_labels)
            print("Test Accuracy: ", accuracy)
            print("Label: ", batch_labels[0].astype(int))
            print("Pred: ", tf_viterbi_sequence[0])